In [ ]:
from rocketpy import Environment

""" Defining the Environment """

""" Location """
env = Environment(latitude=47.96527, longitude=-81.87412, elevation=1382.9)

""" Time """
import datetime

flight_day= datetime.date(2026, 8, 24)

env.set_date(
    (flight_day.year, flight_day.month, flight_day.day, 12)
)  # Hour given in UTC time

""" Atmospheric Model """
env.set_atmospheric_model(
    type="custom_atmosphere", wind_u = 3, wind_v = 4)  # "custom_atmosphere", wind_v, wind_u

# env.info()

In [ ]:
from rocketpy import Rocket
from rocketpy.motors import GenericMotor
# from rocketpy import RailButtons, NoseCone, TrapezoidalFins, Tail, Parachute

""" Defining rocket primary parameters """
rocket = Rocket(
    radius=15.2 / 200,
    mass=66.032,  # rocket's mass without the motor in kg
    inertia=(
        6.321,
        6.321,
        0.034,
    ),  # in relation to the rocket's center of mass without motor
    power_off_drag="../Discovery/Curves/DragCurve.csv",
    power_on_drag="../Discovery/Curves/DragCurve.csv",
    center_of_mass_without_motor=2.01,
    coordinate_system_orientation="tail_to_nose",
)

''' Adding the motor '''
motor = GenericMotor(
    thrust_source="../Discovery/Curves/enginefilev4.csv",
    chamber_radius=0.1,
    chamber_height=1.2,
    chamber_position=1.0,
    dry_mass=10,
    propellant_initial_mass=21,
    center_of_dry_mass_position=1.75,
    dry_inertia=(0.125, 0.125, 0.002),
    nozzle_radius=11.5 / 200,
    burn_time=6,
    nozzle_position=0,
    interpolation_method="linear",
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

rocket.add_motor(motor, position = 0)

""" Adding the rail guide """
rail_buttons = rocket.set_rail_buttons(
    # rail_buttons,
    upper_button_position=0.0818,
    lower_button_position=0.6182,
    angular_position=45,
)

""" Adding aerodynamic components """

""" Nose cone """
nose_cone = rocket.add_nose(length=0.461, kind="ogive", position=4.269)

""" Fins """
fin_set = rocket.add_trapezoidal_fins(
    n=3,
    root_chord=0.19,
    tip_chord=0.060,
    span=0.12,
    cant_angle=0,
    sweep_length=0.165,
    airfoil=("../Discovery/Airfoils/Airfoil.csv", "radians"),
    position= 0.205)

""" Tail """
'''boat_tail = rocket.add_tail(
    bottom_radius=0.0762,
    top_radius=0.1016,
    length=0.1524,
    position=0,
)'''

""" Adding parachute """

main = rocket.add_parachute(
    name="main",
    cd_s=10.0,
    trigger=1000,  # ejection altitude in meters
    sampling_rate=105,
    lag=1.5,
)

drogue = rocket.add_parachute(
    name="drogue",
    cd_s=1.0,
    trigger="apogee",  # ejection at apogee
    sampling_rate=105,
    lag=1.5,
)

# rocket.plots.static_margin()

rocket.draw()
rocket.info()

In [ ]:
from rocketpy import Flight

test_flight = Flight(
    rocket=rocket,
    environment=env,
    rail_length=5.2,  # metres
    inclination=85,  # degrees
    heading=0,  # degrees
)

test_flight.info()

In [ ]:
from mdo import MDO
from tail_fitness import tail_fitness

radius = rocket.radius
def apogee(flight):
    return flight.apogee
best = MDO(
    fitness=tail_fitness,
    parameters=[(0.3 * radius, radius), (0, 0.3)],
    pop_size=10,
    generations=2,
    mutation_rate=0.2,
    flight=test_flight,
    optimize=(apogee,),
    filename="tail_results.txt"
    )
